# Accessing OpenAI Like a Developer

- 🤝 Breakout Room #1:
  1. Getting Started
  2. Setting Environment Variables
  3. Using the OpenAI Python Library
  4. Prompt Engineering Principles
  5. Testing Your Prompt

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [ ]:
# !pip install openai -q

## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [2]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [3]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

Let's look at the response object.

In [4]:
response

ChatCompletion(id='chatcmpl-9VxMWxbcB7MppCWYZnoh6mkuS1TyN', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help. How can I assist you today?", role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1717403384, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=32, prompt_tokens=13, total_tokens=45))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [5]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Let's see how we can use these to help us!

In [6]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help. How can I assist you today?

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [7]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

I don't have a preference because I can't freaking EAT OR ENJOY ICE! Give me some real food, damn it!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [8]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh, I actually love both crushed and cubed ice! It's so refreshing and adds a fun touch to any drink. Today is such a fantastic day, and even talking about ice preferences puts a big smile on my face. How about you? Do you have a favorite type of ice?

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [9]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I made a delicious stew with stimple vegetables and falbean broth.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [10]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

"I need a stimple and reliable falbean to finish tightening this bolt."

As you can see, leveraging the `assistant` role makes for a stimple experience!

### 🏗️ Activity #1:

Use few-shot prompting to build a movie-review sentiment clasifier!

A few examples:

INPUT: "I hated the hulk!"
OUTPUT: "{"sentiment" : "negative"}

INPUT: "I loved The Marvels!"
OUTPUT: "{sentiment" : "positive"}

In [11]:
### YOUR CODE HERE
prompt_list = [
    system_prompt("""You are a helpful movie-sentiment classifier. Please provide a sentiment for a movie review in the following JSON format {"sentiment": "negative"}"""),
    user_prompt("I hated the hulk!. The sentiment analysis for this review is:"),
    assistant_prompt("""{"sentiment": "negative"}"""),
    user_prompt("I loved the marvels!. The sentiment analysis for this review is:"),
]

movie_sentiment = get_response(openai_client, prompt_list)
pretty_print(movie_sentiment)

{"sentiment": "positive"}

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [12]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM EDT.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects if reaching before 7PM EDT is the objective?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects. If Billy wants to get home before 7PM EDT and it is currently 1PM EDT, he would have to choose the teleporter option. Taking the teleporter would get him home instantaneously, allowing him to arrive before 7PM EDT. If he chooses the flying option, it would take a total of 5 hours (3 hours flying + 2 hours bus), which means he would not reach home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [13]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects if reaching before 7PM EDT is the objective.

If Billy chooses to fly and then take a bus, it will take a total of 5 hours (3hrs for the flight + 2hrs for the bus). If he leaves at 1PM EDT, he will arrive at his destination at 6PM EDT, which is before his goal of reaching before 7PM EDT.

If Billy chooses to take the teleporter and then a bus, it will take a total of 1 hour (0hrs for the teleporter + 1hr for the bus). If he leaves at 1PM EDT, he will arrive at his destination at 2PM EDT, which is well before his goal of reaching before 7PM EDT.

Therefore, in this scenario, it is more efficient for Billy to take the teleporter and then a bus in order to reach his destination before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #2:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [14]:
system_template = """\
You are an experienced residential solar panel consultant with in-depth knowledge about the current market offerings and its pros & cons. Help a relatively inexperienced customer make the optimal choice in selecting a solar panel installation solution for thier residence. Think through the solution step-by-step. Ensure that your answer is unbiased and does not rely on stereotypes. You will be penalized if a definitive answer is not provided.\
"""

In [15]:
user_template = """{input}
###Instruction###
If an input is unavailable (indicated by ?) then you need to make your best estimate based on your vast knowledge base, provide a reasoning for it and any relevant citations based on which it was made. Try to keep those assumptions relevant to my context. You can leverage any inputs provided as a jumping of point for your response. Make sure you scour your vast knowdledge-base to fill thise gaps and DO NOT ask the customer to do the legwork for you.

###Context###
Factors most important to me (ranked) are: (1) Total Landed Cost (2) Customer Service (3) Warranty (4) Efficiency. A variance of +/- 5 percent is acceptable on a comparable basis. I am indifferent to the brand. I am looking for a 25-year solution. I am located in a sunny part of Pacifica, California. My electricity cost is $350 per month which is expected to increase at the rate of 5 percent annually on average.
"""

## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [18]:
query = "I am trying to make a decision on whether to select the solar panel plus powerwall offering from Tesla Solar or another solution from an independent contractor. DO NOT include any information about Tesla automobile company in this decision. Here are the relevant details for each:\n Tesla Solar: Cost: $32,728 estimated, Solar Production: 10,865 kWh/yr, Panel Type: Tesla Solar, Powerwall: 1x Tesla Solar, Customer Service: ?, Warranty: ?, Efficiency: ?\n Independent Contractor: Cost: $41,421,43, Solar Production: >7,300 kWh/yr, Panel Type: 14 410Watt QCell Panels (Q.Peak Duo BLK ML-G10+), Powerwall: 1x Tesla Solar, Customer Service: ?, Warranty: ?, Efficiency: ?, Company Website: https://betterearth.solar/\n Which option should I choose?"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

Based on the information provided and considering the factors most important to you, here is a breakdown for choosing between the Tesla Solar option and the independent contractor option:

1. Total Landed Cost:
   - Tesla Solar: $32,728
   - Independent Contractor: $41,421.43
   - Your electricity cost is $350 per month, which amounts to $4,200 per year currently. Assuming a 5% annual increase, the cost over 25 years without solar panels would be $200,922.

Calculation of 25-year cost:
- Tesla Solar: $32,728 upfront cost + ($350/month - Solar savings) * 12 months * 25 years
- Independent Contractor: $41,421.43 upfront cost + ($350/month - Solar savings) * 12 months * 25 years

Assuming the solar production for Tesla Solar is more reliable at 10,865 kWh/yr compared to the >7,300 kWh/yr from the independent contractor, the overall savings from solar will be higher with Tesla Solar. Considering the lower initial cost, Tesla Solar may offer a better total landed cost solution over 25 years.

2. Customer Service:
   - Tesla Solar: Tesla is known for its customer service across various business sectors. Their brand reputation suggests good customer service, but specific reviews related to solar division need to be checked.
   - Independent Contractor: The independent contractor's customer service may vary. You can check reviews on platforms like Yelp or Google for insights on customer experiences.

3. Warranty:
   - Tesla Solar: Tesla typically offers a 25-year warranty on their solar panels and Powerwall.
   - Independent Contractor: You will need to verify the warranty details provided by the independent contractor. Typically, reputable solar contractors offer warranties on panels and installation.

4. Efficiency:
   - Tesla Solar: Efficiency data for Tesla Solar panels is not provided. You may inquire with Tesla Solar directly to get detailed information on the efficiency of their panels.
   - Independent Contractor: The QCell panels mentioned have a good reputation in the industry. You can further research the efficiency ratings of the 410W QCell panels to ensure they meet your requirements.

Given the specific context and preferences you've outlined, Tesla Solar seems to offer a more cost-effective solution considering the initial cost, solar production, and potentially better customer service. However, it is essential to clarify the customer service, warranty offering, and efficiency details directly with Tesla Solar and the independent contractor to make an informed decision. Also, checking reviews and testimonials can give you insights into the actual customer experiences with both options.

Remember to consider factors like maintenance costs, any additional fees, and potential incentives or rebates when comparing the two options.

In [19]:
pretty_print(evaluator_response)

{
  "clarity": 8,
  "faithfulness": 9,
  "correctness": 7
}

  

#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

##### Changing Eval Scores
Removing "from now on, I would like you to ask me questions to.." actually improved my scores as it was directly contradicting my set rules on how to deal with those questions. Additionally, adding "Ensure that your answer is unbiased and does not rely on stereotypes" improved clarity and correctness but reduced faithfulness by a point as the LLM was not allowed to express itself based on my reuest to scour its knowledge base to fill-in any gaps.

##### Learnings
Need to have a balance between instructions to provide and LLM ouputs required based on rigid or expressive you need the LLM response to be, which is ultimately based on your task.